# Als로 돌리는 파일 
* 이 코드 돌릴땐 scaled_r(정규화된 통합데이터), scaled_not(정규화된 explicit데이터)으로 돌리기

In [164]:
import sklearn
import random 

# Pyspark Library #
# SQL
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import mean, col, split, regexp_extract, when, lit
# ML
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler, IndexToString
from pyspark.ml.feature import QuantileDiscretizer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

import os
import pandas as pd
import numpy as np
os.chdir('/Users/duduuman/Documents/last/')

# 스파크 세션 만들기
spark = SparkSession\
        .builder\
        .appName('als')\
        .getOrCreate()

df = spark.read.csv(os.getcwd() + '/scaled_r.csv',
                   inferSchema=True, header=True)
df.limit(3).toPandas()

,_c0,user_id,product_id,rating
0,0,4,72,0.666667
1,1,4,69,0.666667
2,2,938,71,0.666667


In [165]:
df = df[['user_id', 'product_id', 'rating']] 
df['rating'] 

Column<'rating'>

In [182]:
# ALS recommender algorithm
from pyspark.ml.recommendation import ALS

train, test = df.randomSplit([0.75, 0.25])

rec = ALS(maxIter=10,
         regParam=0.01,
         userCol='user_id',
         itemCol='product_id',
         ratingCol='rating', # label -> predict할 때는 필요 없음!
         nonnegative=True,
         numUserBlocks=10,
         numItemBlocks=10, 
         alpha = 10000,
         coldStartStrategy='drop')
# ALS모델 학습 -> dataframe을 넣어주기
rec_model = rec.fit(train)

# transform을 이용해 예측 -> dataframe을 넣어주기
pred_ratings = rec_model.transform(test)
pred_ratings.limit(5).toPandas()

,user_id,product_id,rating,prediction
0,14832,57,0.333333,0.409758
1,28146,164,0.666667,0.456597
2,32855,127,0.333333,0.540217
3,32855,134,0.666667,0.590608
4,32855,217,0.666667,0.660398


In [183]:
# Get metric for training
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(labelCol='rating',
                               predictionCol='prediction',
                               metricName='rmse')
# evaluate 메소드에 예측값 담겨있는 dataframe 넣어주기
rmse = evaluator.evaluate(pred_ratings)

mae_eval = RegressionEvaluator(labelCol='rating',
                              predictionCol='prediction',
                              metricName='mae')
mae = mae_eval.evaluate(pred_ratings)

print("RMSE:", rmse)
print("MAE:", mae)

RMSE: 0.18036382382920535
MAE: 0.14134375887471412
